In [33]:
import numpy as np

In [ ]:
np.random.seed(0)
data = -2 * np.random.rand(100,2)
data1=1+2*np.random.rand(50,2)
data[50:100,:]=data1

In [ ]:
data

array([[-1.09762701, -1.43037873],
       [-1.20552675, -1.08976637],
       [-0.8473096 , -1.29178823],
       [-0.87517442, -1.783546  ],
       [-1.92732552, -0.76688304],
       [-1.58345008, -1.05778984],
       [-1.13608912, -1.85119328],
       [-0.14207212, -0.1742586 ],
       [-0.04043679, -1.66523969],
       [-1.5563135 , -1.7400243 ],
       [-1.95723668, -1.59831713],
       [-0.92295872, -1.56105835],
       [-0.23654885, -1.27984204],
       [-0.28670657, -1.88933783],
       [-1.04369664, -0.82932388],
       [-0.52911122, -1.54846738],
       [-0.91230066, -1.1368679 ],
       [-0.0375796 , -1.23527099],
       [-1.22419145, -1.23386799],
       [-1.88749616, -1.3636406 ],
       [-0.7190158 , -0.87406391],
       [-1.39526239, -0.12045094],
       [-1.33353343, -1.34127574],
       [-0.42076512, -0.2578526 ],
       [-0.6308567 , -0.72742154],
       [-1.14039354, -0.87720303],
       [-1.97674768, -0.20408962],
       [-0.41775351, -0.32261904],
       [-1.30621665,